In [1]:
import argparse
from datetime import datetime, date
from uti import DataLoader, Logger
from Database import GSDatabase, GSPriceDf
import re
from Model.settings import DataCleaner
import pandas as pd
import numpy as np
from datetime import datetime, date

GSD = GSDatabase()
logger = Logger()
DL = DataLoader()
DC = DataCleaner()

Current root directory: C:/Users/JayYang/OneDrive - Alpha Sherpa Capital
Current working directory C:\Users\JayYang\Platform\Goldman


2022-05-11 16:37:31,115 P[23700] [MainThread 23436] Error: no proxy address identified.
Check if Eikon Desktop or Eikon API Proxy is running.
2022-05-11 16:37:31,119 P[23700] [MainThread 23436] Error on handshake url http://127.0.0.1:None/api/handshake : UnsupportedProtocol("Request URL is missing an 'http://' or 'https://' protocol.")
2022-05-11 16:37:31,121 P[23700] [MainThread 23436] Error on handshake url http://127.0.0.1:None/api/handshake : UnsupportedProtocol("Request URL is missing an 'http://' or 'https://' protocol.")
2022-05-11 16:37:31,123 P[23700] [MainThread 23436] Port number was not identified, cannot send any request


Engine current settings: 
daily_position 18 
region_position 6 
skewness: 0


In [ ]:
"""
1. Get train data before march, used as the scoring training data on reports after march

2. Pull the reports after march, 
- Apply before march scoring on march reports
- Apply before april scoring on april reports
- Apply scoring for Goldman and Citi

3. From "Jay pnl for benchmark.csv", calculate the number of longs/shorts on each day; real trades pnl, times ATR(x) to reach the 
pnl as is the case where ATR(x)=1 is used in system calculation

4. Select trades from Goldman/Citi based on daily numbers of shorts/longs
Three cases to consider
- Wholly Goldman
- Wholly Citi
- Randomly from Goldman/Citi

"""

In [2]:
train_data, test_data = DC.get_benchmark_test_data()

16:38:09 C:/Users/JayYang/OneDrive - Alpha Sherpa Capital/Goldman/Database\Log/20220511/20220511163732.txt created.
16:38:09 Loading C:/Users/JayYang/OneDrive - Alpha Sherpa Capital/Goldman/Database\Backtest/Benchmark data/X_train.csv
16:38:09 Loading C:/Users/JayYang/OneDrive - Alpha Sherpa Capital/Goldman/Database\Backtest/Benchmark data/X_test.csv


In [3]:
all_data = pd.concat([train_data, test_data], axis=0)

In [6]:
from Model import DataCleaner, benchmark_expectancy

In [ ]:
# Get Citi Reports with scores for March

In [7]:
train_data_before_mar = all_data[all_data['Date'] < datetime(2022, 3, 1)].copy()
test_data_mar = all_data[(all_data['Date'] >= datetime(2022, 3, 1)) & (all_data['Date'] < datetime(2022, 4, 1))].copy()
test_data_mar

,Date,Time,Ticker,Source,Asset Class,Headline,Summary,Analysts,Report Type,Headline sentiment,...,release_period,ticker_updated,gap_in_atr,market_cap_grp,No. of trades,side,stop_loss,d0_r,d1_r,d2_r
723,2022-03-31,2022-03-31 15:44:00,RH,Research,Equity,RH (RH): Geopolitical and macroeconomic volati...,Revenue increased +10.9% y/y to $901.5mn (belo...,Kate McShane| Mark Jordan| Leah Jordan| Tan...,Target Price Decrease,negative,...,Before,RH US,-1.324684,MidCap,1,short,353.542592,0.130081,0.358546,-0.065864
724,2022-03-31,2022-03-31 11:46:00,FLEX,Research,Equity,Flex (FLEX): Key takeaways from investor day; ...,Flex hosted a virtual investor day on 3/30/22 ...,Mark Delaney| Bruno Dossena| Ryan Heeb| Ele...,Estimate Change,positive,...,After,FLEX US,-0.126097,MidCap,1,long,17.334175,1.555195,1.471130,0.861662
725,2022-03-31,2022-03-31 00:18:00,600690.SS,Research,Equity,Haier Smart Home Co. (600690.SS): 2021 First T...,Haier Smart Home reported an in-line good set ...,Nicolas Yi| Cecilia Tang,Earning's Review,positive,...,Before,600690.SS CH,0.320368,LargeCap,1,long,22.229648,0.115333,0.948289,0.781698
726,2022-03-31,2022-03-31 00:47:00,TELIA.ST,Research,Equity,Telia Co. (TELIA.ST): Data Update: Minor chang...,We make minor changes to estimates. We do not ...,Andrew Lee| Nicola Gifford| Yemi Falana| Ha...,Estimate Change,neutral,...,After,TELIA.ST SS,0.049395,LargeCap,1,neutral,0.000000,0.000000,0.000000,0.000000
727,2022-03-31,2022-03-31 19:42:00,VNET,Research,Equity,21Vianet Group Inc. (VNET): Management cuts 20...,21Vianet reported its 4Q21 results on Mar 30 a...,Tina Hou| Hao Chen,Estimate Change,negative,...,Before,VNET US,-0.078829,SmallCap,1,short,7.561146,1.274395,0.814562,-0.249624
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1974,2022-03-01,2022-03-01 10:02:00,ENDP,Research,Equity,Endo International Plc (ENDP): First Take: Sof...,ENDP's 4Q21 revenue exceeded GS/consensus (Fac...,Nathan Rich| Lindsay Golub| Sarah Conrad,Estimate Change,negative,...,After,ENDP US,0.000000,SmallCap,1,short,3.061855,1.811481,2.415308,1.984003
1975,2022-03-01,2022-03-01 00:54:00,UTOS.SI,Research,Equity,UOL Group (UTOS.SI): 4Q21 review: Revenue impr...,UOL reported FY21 Patmi of S$307m and core Pat...,Xuan Tan| Tyler Atipas,Earning's Review,positive,...,Before,UTOS.SI SP,0.071787,MidCap,1,long,6.890699,-0.071787,-0.933234,-0.933234
1976,2022-03-01,2022-03-01 00:31:00,PKO.WA,Research,Equity,PKO Bank Polski (PKO.WA): Data Update: Incorpo...,We have incorporated PKO BP's FY21 results and...,Mikhail Butkov,Estimate Change,neutral,...,After,PKO.WA PW,-0.407163,LargeCap,1,neutral,0.000000,0.000000,0.000000,0.000000
1977,2022-03-01,2022-03-01 00:44:00,0880.HK,Research,Equity,SJM Holdings (0880.HK): 4Q21 results below exp...,SJM reported -HK$611mn EBITDA loss in 4Q21 sli...,Simon Cheung| Alpha Wang| Leah Pan| Peter Y...,Target Price Decrease,negative,...,Before,0880.HK HK,0.000000,MidCap,1,short,4.564947,0.634311,1.268623,1.122243


In [8]:
test_data_mar = benchmark_expectancy(sort_by='exch_region', train_data=train_data_before_mar, test_data=test_data_mar)

16:49:55 Loading C:/Users/JayYang/OneDrive - Alpha Sherpa Capital/Goldman/Database/Backtest/Benchmark data/blind_long.csv
16:49:56 Loading C:/Users/JayYang/OneDrive - Alpha Sherpa Capital/Goldman/Database/Backtest/Benchmark data/blind_short.csv
16:49:57
           Date                Time     Ticker    Source Asset Class  \
0    2022-03-31 2022-03-31 00:18:00  600690.SS  Research      Equity   
1    2022-03-31 2022-03-31 19:34:00  145020.KQ  Research      Equity   
2    2022-03-31 2022-03-31 23:25:00    2319.HK  Research      Equity   
3    2022-03-31 2022-03-31 07:56:00    2216.HK  Research      Equity   
4    2022-03-31 2022-03-31 00:12:00    1288.HK  Research      Equity   
...         ...                 ...        ...       ...         ...   
1973 2022-03-01 2022-03-01 00:12:00    3868.HK  Research      Equity   
1974 2022-03-01 2022-03-01 10:02:00       ENDP  Research      Equity   
1975 2022-03-01 2022-03-01 00:54:00    UTOS.SI  Research      Equity   
1976 2022-03-01 2022-03-01

In [ ]:
# Get Citi Reports with scores for April

In [9]:
train_data_before_apr = all_data[all_data['Date'] < datetime(2022, 4, 1)].copy()
test_data_apr = all_data[(all_data['Date'] >= datetime(2022, 4, 1)) & (all_data['Date'] < datetime(2022, 5, 1))].copy()
test_data_apr

,Date,Time,Ticker,Source,Asset Class,Headline,Summary,Analysts,Report Type,Headline sentiment,...,release_period,ticker_updated,gap_in_atr,market_cap_grp,No. of trades,side,stop_loss,d0_r,d1_r,d2_r
0,2022-04-29,2022-04-29 01:30:00,603288.SS,Research,Equity,Foshan Haitian Flavouring & Food (603288.SS): ...,Haitian reported 1Q22 results on Apr 28th. The...,Leaf Liu| Yutong Zhou| Brian Dai,Earning's Review,negative,...,Before,603288.SS CH,-1.742313,LargeCap,1,short,87.025689,0.165055,0.000000,0.000000
1,2022-04-29,2022-04-29 01:54:00,600406.SS,Research,Equity,Nari Tech Co. (600406.SS): FY21 results inline...,Nari's FY21 results was inline with profit ale...,Chao Ji| Chelsea Zhai| Yawen Wang,Earning's Review,positive,...,Before,600406.SS CH,0.635854,LargeCap,1,long,28.651586,1.108023,0.000000,0.000000
2,2022-04-29,2022-04-29 02:56:00,002271.SZ,Research,Equity,Oriental Yuhong (002271.SZ): NDR takeaways: Po...,Key takeaways from GS-hosted post-results NDR ...,Nick Zheng| Carrie Jiang| Cayle Song,Estimate Change,positive,...,Before,002271.SZ CH,0.215001,LargeCap,1,long,42.610942,0.201838,0.000000,0.000000
3,2022-04-29,2022-04-29 05:18:00,RMD.AX,Research,Equity,ResMed Inc. (RMD.AX): First Take: 3Q22 results...,Key takeaways Group revenue misses -5% driven ...,Chris Cooper| Bosco Feng,Earning's Review,positive,...,Before,RMD.AX AU,-0.905822,LargeCap,1,long,28.528507,-1.000000,-1.000000,-1.000000
4,2022-04-29,2022-04-29 07:38:00,601021.SS,Research,Equity,Spring Airlines Co. (601021.SS): FY21 and 1Q22...,Spring Airlines reported 2021 full-year and 1Q...,Herbert Lu| Simon Cheung| Leah Pan,Earning's Review,positive,...,Before,601021.SS CH,0.015801,MidCap,1,long,43.691326,0.205407,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
718,2022-04-01,2022-04-01 01:54:00,AKZO.AS,Research,Equity,Akzo Nobel (AKZO.AS): Revising estimates ahead...,We update our estimates for Akzo Nobel ahead o...,Georgina Fraser| Ji woo Song| Isabel Appen,Earning's Review,positive,...,After,AKZO.AS NA,0.145796,LargeCap,1,long,76.053613,-0.015347,0.291591,-0.122775
719,2022-04-01,2022-04-01 00:54:00,1COV.DE,Research,Equity,Covestro (1COV.DE): Revising estimates ahead o...,We update our estimates ahead of Covestro's 1Q...,Georgina Fraser| Ji woo Song| Isabel Appen,Estimate Change,neutral,...,After,1COV.DE GY,0.197563,MidCap,1,neutral,0.000000,0.000000,0.000000,0.000000
720,2022-04-01,2022-04-01 02:51:00,AOXG.DE,Research,Equity,Alstria Office Reit AG (AOXG.DE): Terminating ...,Following the acquisition of the company by Br...,Jonathan Kownator| Anjali Nath| Sheroz Zia,Estimate Change,neutral,...,After,AOXG.DE GY,-0.049383,MidCap,1,neutral,0.000000,0.000000,0.000000,0.000000
721,2022-04-01,2022-04-01 01:16:00,1813.HK,Research,Equity,KWG Group (1813.HK): First Take: FY21 a substa...,KWG reported FY2021 unaudited results with cor...,Vicky Li| Yi Wang| Shi Xu| Mengwen Wang| an...,Earning's Review,negative,...,Before,1813.HK HK,-0.385664,SmallCap,1,short,3.513010,-0.303022,-1.000000,-1.000000


In [10]:
test_data_apr = benchmark_expectancy(sort_by='exch_region', train_data=train_data_before_apr, test_data=test_data_apr)

16:50:59 Loading C:/Users/JayYang/OneDrive - Alpha Sherpa Capital/Goldman/Database/Backtest/Benchmark data/blind_long.csv
16:50:59 Loading C:/Users/JayYang/OneDrive - Alpha Sherpa Capital/Goldman/Database/Backtest/Benchmark data/blind_short.csv
16:51:01
          Date                Time     Ticker    Source Asset Class  \
0   2022-04-29 2022-04-29 01:54:00  600406.SS  Research      Equity   
1   2022-04-29 2022-04-29 02:56:00  002271.SZ  Research      Equity   
2   2022-04-29 2022-04-29 05:18:00     RMD.AX  Research      Equity   
3   2022-04-29 2022-04-29 07:38:00  601021.SS  Research      Equity   
4   2022-04-28 2022-04-28 08:34:00  688521.SS  Research      Equity   
..         ...                 ...        ...       ...         ...   
717 2022-04-01 2022-04-01 11:09:00         BG  Research      Equity   
718 2022-04-01 2022-04-01 01:54:00    AKZO.AS  Research      Equity   
719 2022-04-01 2022-04-01 00:54:00    1COV.DE  Research      Equity   
720 2022-04-01 2022-04-01 02:51:00  

In [11]:
results = pd.concat([test_data_mar, test_data_apr], axis=0)

In [13]:
results['Broker'] = 'Goldman'

In [14]:
DL.toDB(results, 'Goldman system reports.csv')